<a href="https://colab.research.google.com/github/vkthakur90/jupyter_notebooks/blob/master/LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain

In [2]:
!pip install huggingface_hub

In [3]:
!pip install python-dotenv

In [4]:
import langchain as lc
import google.colab.files as gcf
import numpy as np
import pandas as pd
import dotenv as denv

# **Uploading environment files containing huggingface access token**

In [5]:
!ls

uploaded = gcf.upload()

for k, v in uploaded.items():
    with open(k, 'wb') as f:
        f.write(v)

!ls

hf.env	sample_data


hf.env	sample_data


# **Mention repository identity of hosted model**

In [6]:
REPO_ID = 'google/flan-t5-xxl'

In [7]:
denv.load_dotenv('hf.env')

True

In [8]:
hf_google_flan_t5_xxl = lc.HuggingFaceHub(repo_id = REPO_ID)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


# **Create prompt template**

In [9]:
template = '''/
You are a vehicle mechanic, give responses to the following/
question: {question}. Do not use technical words, give easy/
to understand responses. Your response should be in {language}
'''

In [10]:
prompt = lc.PromptTemplate(
    template = template,
    input_variables=["question", "language"]
)

In [11]:
chain = lc.LLMChain(
    llm = hf_google_flan_t5_xxl,
    prompt = prompt
)

In [14]:
chain.run({
    'question' : "Why won't a vehicle start on ignition?",
    'language' : "English"
})

'The battery is dead.'